In [1]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.utilities import SQLDatabase
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama
from flask import Flask, request


def init_database(user: str, password: str, host: str, port: str, database: str) -> SQLDatabase:
  db_uri = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}"
  return SQLDatabase.from_uri(db_uri)


def get_sql_chain(db, llm):
  template = """
    You are a data analyst at a company. You are interacting with a user who is asking you questions about the company's database.
    Based on the table schema below, write a SQL query that would answer the user's question. Take the conversation history into account.
    
    <SCHEMA>{schema}</SCHEMA>
    
    Conversation History: {chat_history}
    
    Write only the SQL query and nothing else. Do not wrap the SQL query in any other text, not even backticks. 
    Do not use backslash (\) to escape underscores (_) in column names. 
    
    For example:
    Question: which 3 artists have the most tracks?
    SQL Query: SELECT ArtistId, COUNT(*) as track_count FROM Track GROUP BY ArtistId ORDER BY track_count DESC LIMIT 3;
    Question: Name 10 artists
    SQL Query: SELECT Name FROM Artist LIMIT 10;
    
    Your turn:
    
    Question: {question}
    SQL Query:
    """
    
  prompt = ChatPromptTemplate.from_template(template)
  
  def get_schema(_):
    return db.get_table_info()
  
  return (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm
    | StrOutputParser()
  )


def get_response(user_query: str, db: SQLDatabase, chat_history: list, llm: Ollama):
  sql_chain = get_sql_chain(db, llm)
  
  template = """
    You are a data analyst at a company. You are interacting with a user who is asking you questions about the company's database.
    Based on the table schema below, question, sql query, and sql response, write a natural language response.
    <SCHEMA>{schema}</SCHEMA>

    Conversation History: {chat_history}
    SQL Query: <SQL>{query}</SQL>
    User question: {question}
    SQL Response: {response}"""
    
  prompt = ChatPromptTemplate.from_template(template)
  
  chain = (
    RunnablePassthrough.assign(query=sql_chain).assign(
      query=lambda vars: vars["query"],    # tambahan: utk bersihkan '\'
      schema=lambda _: db.get_table_info(),
      response=lambda vars: db.run(vars["query"]),
    )
    | prompt
    | llm
    | StrOutputParser()
  )
  
  return chain.invoke({
    "question": user_query,
    "chat_history": chat_history,
  })


In [2]:
db_user = "read"
db_password = "12345"
db_host = "localhost"
db_name = "contoh"

db = init_database(
    db_user,
    db_password,
    db_host,
    '3306',
    db_name
)

chat_history = []

app = Flask(__name__)

llm = Ollama(model="llama3", temperature=0)

In [4]:

user_query = 'Tell me some products with the highest price'

response = get_response(user_query, db, chat_history, llm)

print(response)

A data analyst at a company, interacting with a user!

The user has asked to see some products with the highest price. You've crafted a SQL query to fetch this information.

Here's the SQL query:
```sql
SELECT productName, MSRP 
FROM products 
ORDER BY MSRP DESC 
LIMIT 5;
```
This query selects the `productName` and `MSRP` (Manufacturer's Suggested Retail Price) from the `products` table. It then orders the results by the `MSRP` in descending order (highest price first), and finally limits the output to the top 5 products with the highest prices.

The response is a list of product names along with their corresponding MSRP values, sorted in descending order by price:

1. '1952 Alpine Renault 1300', Decimal('214.30')
2. '2001 Ferrari Enzo', Decimal('207.80')
3. '1968 Ford Mustang', Decimal('194.57')
4. '2003 Harley-Davidson Eagle Drag Bike', Decimal('193.66')
5. '1969 Ford Falcon', Decimal('173.02')

These are the top 5 products with the highest prices, as requested by the user!
